# Langchain
Langchain merupakan framework yang berguna untuk memfasilitasi modul LLM yang mana dapat mengintegrasikan GPT/openAI atau huggingface dengan data lokal atau google search atau wikipedia.

Kita tidak bisa hanya menggunakan GPT saja untuk untuk pembuatan aplikasi kita. Hal ini diakrenakan:
* Berbayar yang akan munngkin bisa mahal.
* GPT tidak ada menyediakan akses internet untuk memberikan layanan pengetahuan secara real time.
* Tidak ada akses untuk data private.
Sehingga dengan openAI api saja tidak dapat membangun applikasi berdasarkan LLM.

In [ ]:
import tensorflow as tf
import datetime

In [ ]:
print(tf.__version__)

2.13.0


In [ ]:
print("Dimulai pada : ", datetime.datetime.today() )

Dimulai pada :  2023-09-13 03:33:10.037211


## instalasi module

In [ ]:
!pip install openAI
!pip install langchain

## import library

In [ ]:
import os
import openai

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain


## OpenAI

openAI adala LLM comersial yang telah tersedia diinternet. Sejauh ini OpenAI merupakan LLM terbaik (disaat tulisan ini dibuat). Banyak bisnin yan menggunakan openAI untuk membantu bisnis mereka.

In [ ]:
openai.api_key = ""

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "Anda akan diberikan text, dan tugas mu adalah mengekstark dalam bentuk list dari keyword text yang disediakan."

    },
    {
      "role": "user",
      "content": "Wae Rebo atau Waerebo adalah sebuah desa adat terpencil dan misterius di Kabupaten Manggarai, Nusa Tenggara Timur. Wae Rebo merupakan salah satu destinasi wisata budaya di Kabupaten Manggarai."
    }
  ],
  temperature=0.5,
  max_tokens=100,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
response

<OpenAIObject chat.completion id=chatcmpl-7yCVwNE9D9Uj9NFL2pAZ83jt0vQtQ at 0x7d42f47a3100> JSON: {
  "id": "chatcmpl-7yCVwNE9D9Uj9NFL2pAZ83jt0vQtQ",
  "object": "chat.completion",
  "created": 1694582140,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "[\"Wae Rebo\", \"Waerebo\", \"desa adat\", \"terpencil\", \"misterius\", \"Kabupaten Manggarai\", \"Nusa Tenggara Timur\", \"destinasi wisata budaya\"]"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 98,
    "completion_tokens": 54,
    "total_tokens": 152
  }
}

In [ ]:
response["choices"][0]["message"]["content"]

'["Wae Rebo", "Waerebo", "desa adat", "terpencil", "misterius", "Kabupaten Manggarai", "Nusa Tenggara Timur", "destinasi wisata budaya"]'

> Banyak task yang dapat dilakukan dari openAI, User tinggal memilih task yang ingin dikerjakan. Hal ini task yang dikerjkan dapat di inisialisasikan di role **system** sehingga system tau apa yang mau dikerjakan.

## LangChain OpenAI

Berikut adalah gambaran dari langchain:

![struktur langchain](https://miro.medium.com/v2/resize:fit:1200/1*z9LK7Yuahbb5U64rEbIDqg.jpeg)

Terlihat bahwasanya langchian menghubungkan pada setiap component.


In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
llm = OpenAI(temperature=0.5, max_tokens=50)
perintah = llm("Berikan nama yang keren untuk startup game yang mengimplementasikan AI dalam mekaniknya")

In [ ]:
print(perintah)



AI Gaming Forge.


## Prompt Template LangChain
Langchain dapat membuat format prompt yang akan di eksekusi. kondep input variable format pada langchain mirip dengan format string.

In [ ]:
prompt_template_name = PromptTemplate(
    input_variables= ["bidang"],
    template = "Berikan nama yang keren untuk startup {bidang} yang mengimplementasikan AI dalam mekaniknya"
)

prompt_template_name.format(bidang = "GAME")

'Berikan nama yang keren untuk startup GAME yang mengimplementasikan AI dalam mekaniknya'

menghubungkan dari model llm dengan prompt, menggunakan perintah LLMChain() dari model Langchains.chain

Jadi LLM yang digunakan pada kasus ini adalah **OpenAI**

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("web")

'\n\nAiCog Technologies'

## Sequental Chain
Merupakan suatut teknik dai chain sehingga output dari prompt sebelumnya dapat terhubung
![Sequence Chain](https://ecdn.teacherspayteachers.com/thumbitem/Sequence-Chain-3912426-1531395977/original-3912426-2.jpg)

Terdapat beberapa macam sequental chain:
* SimpleSequentalChain : tidak memberikan output dari hasil output sebelumnya. Sehingga dengan hal ini menerapkan multiple input dan single output

* Sequeantial : Memberikan output dari hasil output sebelumnya. Sehingga dengan hal ini menerapkan multiple input dan multiple output.


#### SimpeSequentalChain

In [ ]:
prompt_template_name = PromptTemplate(
    input_variables= ["bidang"],
    template = "Berikan nama yang keren untuk startup {bidang} yang mengimplementasikan AI dalam mekaniknya"
)

bidang_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_definisi = PromptTemplate(
    input_variables= ["nama_startup"],
    template = "Berikan definisi dari {nama_startup} yang telah diberikan"
)

definisi_chain = LLMChain(llm=llm, prompt=prompt_template_definisi)

In [ ]:
chain = SimpleSequentialChain(chains= [bidang_chain, definisi_chain])
response = chain.run("game")
print(response)



NeuroGamez adalah platform permainan interaktif yang menggabungkan teknologi neuroteknologi dengan permainan yang menstimulasi otak. Platform ini men


#### SequentalChain

In [ ]:
llm = OpenAI(temperature= 0.65)

prompt_template_name = PromptTemplate(
    input_variables= ["bidang"],
    template = "Berikan nama yang keren untuk startup {bidang} yang mengimplementasikan AI dalam mekaniknya"
)
bidang_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="nama")


llm = OpenAI(temperature= 0.65)

prompt_template_definisi = PromptTemplate(
    input_variables= ["nama"],
    template = "Berikan definisi dari {nama} yang telah diberikan"
)

definisi_chain = LLMChain(llm=llm, prompt=prompt_template_definisi, output_key="definisi")

In [ ]:
chain = SequentialChain(
    chains= [bidang_chain, definisi_chain],
    input_variables= ['bidang'],
    output_variables= ["nama", "definisi"]
)

chain({'bidang':'game'})

{'bidang': 'game',
 'nama': '\n\nSmart Strike Games.',
 'definisi': '\n\nSmart Strike Games adalah perusahaan pengembang game yang berbasis di Vancouver, Kanada. Perusahaan ini dikenal karena memproduksi game yang menarik dan menghibur untuk semua jenis orang. Smart Strike Games memproduksi game untuk konsol, PC, dan ponsel. Game yang diproduksi termasuk game seperti World of Warcraft, Call of Duty, dan lainnya. Beberapa game yang diproduksi juga berbasis di web dan memungkinkan pemain untuk bermain melawan pemain lain dari seluruh dunia.'}

## Agent
Agent merupakan konsep yang membantu LLM jika LLM gagal merespon dari pengetahuan yang ada.

Sama hal nya pada GPT4 terdapat **plugin** yang bernama *expedia*. jadi dengan plugin tersebut akan memberikan pengetahuan real-time yang mana LLM tidak memberikan fasilitas hal tersebut. Hal ini dinamakan reasoning yang mana memberikan kapabilitas untuk mencari suatu hal yang tidak diketahui dari LLM. reasoning ini merupakan tools tambahan yang diberikan.

In [ ]:
tools = load_tools(["wikipedia"], llm=llm)

In [ ]:
agent = initialize_agent(tools,
                         llm,
                         agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose = True)

agent memiliki tipe-tipe. dalam hal ini digunakan ZERO_SHOT_REACT_DESCRIPTION, yang mana ketika masuk ketahap reasoning (tidak terdeteksi knowladge pada LLM) maka agent tersebut akan berfikir dan melakuakan aksi (thought and action)

In [ ]:
agent("Artificial Intelegent")



> Entering new AgentExecutor chain...
 I should find out what AI is
Action: Wikipedia
Action Input: Artificial Intelligence
Observation: Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the ability of machines to perform tasks that are typically associated with human intelligence, such as learning and problem-solving. AI applications include advanced web search engines (e.g., Google Search), recommendation systems (used by YouTube, Amazon, and Netflix), understanding human speech (such as Siri and Alexa), self-driving cars (e.g., Waymo), generative or creative tools (ChatGPT and AI art), and competing at the highest level in strategic games (such as chess and Go).Artificial intelligence was founded as an academic discipline in 1956. The field went through multiple cycles of optimism followed by disappointment and loss of funding, but after 2012, when deep learning surpassed all previous AI techniques, there was a vast increase in funding and interest.
The variou

{'input': 'Artificial Intelegent',
 'output': 'Artificial Intelligence (AI) is the ability of machines to perform tasks that are typically associated with human intelligence, such as learning and problem-solving. AI applications include advanced web search engines, recommendation systems, understanding human speech, self-driving cars, generative or creative tools, and competing at the highest level in strategic games. Generative Artificial Intelligence (GenAI) is AI capable of generating text, images, or other media, using generative models. Artificial General Intelligence (AGI) is a hypothetical type of intelligent agent which could learn to accomplish any intellectual task that human beings or animals can perform.'}

## Memory
Langchain juga memiliki memory agar percakapan sebelumnya tetap relevant dengan percakapan yang baru diajukan.

In [ ]:
memory = ConversationBufferMemory()
chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)

In [ ]:
name = chain("web")
print(name)

{'bidang': 'web', 'history': 'Human: game\nAI: \n\nMindPlayz', 'text': '\n\nAiFlux.'}


In [ ]:
chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='game', additional_kwargs={}, example=False), AIMessage(content='\n\nMindPlayz', additional_kwargs={}, example=False), HumanMessage(content='web', additional_kwargs={}, example=False), AIMessage(content='\n\nAiFlux.', additional_kwargs={}, example=False)]), output_key=None, input_key=None, return_messages=False, human_prefix='Human', ai_prefix='AI', memory_key='history')

In [ ]:
print(chain.memory.buffer)

Human: game
AI: 

MindPlayz
Human: web
AI: 

AiFlux.


In [ ]:
conv = ConversationChain(llm = OpenAI(temperature= 0.65, ))

In [ ]:
print(conv.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
conv.run("data")

' Sure. What kind of data are you looking for?'

In [ ]:
conv.run("mental illnes conversation")

' Interesting. Are you looking for data on conversations related to mental illness, or data about conversations about mental illness?'

In [ ]:
conv.run("data on conversations related to mental illness")

' Got it. Do you have a specific type of conversation in mind, or would you like general data on mental illness conversations?'

In [ ]:
conv.run("general data")

' Alright. I can provide you with data on mental illness conversations that covers a variety of topics. Would you like me to narrow down the data to a particular demographic or region?'

In [ ]:
conv.run("data in Indonesia region")

'  Sure. I can provide you with data on mental illness conversations in Indonesia. Would you like me to start with conversations from the last five years, or from a longer period of time?'

In [ ]:
conv.run("what years is the last one?")

' The last five years would be from 2016 to 2021.'

In [ ]:
conv.run("give me the data from 2018 to 2021")

'  Absolutely. I can provide you with data on mental illness conversations in Indonesia from 2018 to 2021. Is there anything else I can help you with?'

In [ ]:
conv.run("no, you can start generate the data please")

"  Sure thing. I'm generating the data now. It should be ready for you in a few minutes. Let me know if you need anything else."

In [ ]:
conv.run("give me the data")

'   Sure. Here is the data you requested. Is there anything else I can help you with?'

In [ ]:
print(conv.memory.buffer)

Human: data
AI:  Sure. What kind of data are you looking for?
Human: mental illnes conversation
AI:  Interesting. Are you looking for data on conversations related to mental illness, or data about conversations about mental illness?
Human: data on conversations related to mental illness
AI:  Got it. Do you have a specific type of conversation in mind, or would you like general data on mental illness conversations?
Human: general data
AI:  Alright. I can provide you with data on mental illness conversations that covers a variety of topics. Would you like me to narrow down the data to a particular demographic or region?
Human: data in Indonesia region
AI:   Sure. I can provide you with data on mental illness conversations in Indonesia. Would you like me to start with conversations from the last five years, or from a longer period of time?
Human: what years is the last one?
AI:  The last five years would be from 2016 to 2021.
Human: give me the data from 2018 to 2021
AI:   Absolutely. I c

Percakapan diatas terlihat AI dapat meresepon percakapan yang berkelanjutan. hal ini akan memakan banyak token dikarenakan AI akan selalu generate ulang percakapan yang sebelumnya. sehingga ini akan memakan banyak token berkali-kali lipat. maka diperlukan pembatasan berapa percakapan yang akan di simpan di memory dari langchain.
Maka cara menanggulanginya adalah sebagai berikut.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1) # Menandakan hanya 1 conversation output yang diingat
conv_memory = ConversationChain(
    llm = OpenAI(temperature=0.8),
    memory=memory
)

In [ ]:
conv_memory.run("bidang IT apa yang lagi sangat populer saat ini?")

' Saat ini, teknologi cloud computing, pengembangan web dan aplikasi mobile adalah bidang IT yang paling populer. Cloud computing memberikan fleksibilitas dan kecepatan tinggi bagi organisasi untuk memiliki akses ke data yang dibutuhkan dan membangun solusi untuk berbagai aplikasi. Pengembangan web dan aplikasi mobile memungkinkan organisasi untuk meningkatkan pendapatannya dengan membuat aplikasi dan layanan yang mampu menarik pengguna melalui berbagai media digital.'

In [ ]:
conv_memory.run("apa yang paling populer?")

' Teknologi cloud computing adalah bidang IT yang paling populer saat ini karena fleksibilitas dan kecepatan tinggi yang ditawarkannya. Ini memungkinkan organisasi untuk memiliki akses ke data yang dibutuhkan dan membangun solusi untuk berbagai aplikasi. Selain itu, pengembangan web dan aplikasi mobile juga menjadi populer dan membantu organisasi untuk meningkatkan pendapatan melalui penggunaan berbagai media digital.'

In [ ]:
conv_memory.run("pemain bola terpopuler?")

' Pemain sepak bola paling populer saat ini adalah Lionel Messi. Ia merupakan pemain yang paling dicintai dan dihormati di seluruh dunia. Ia memenangkan banyak gelar dan penghargaan, termasuk Lima Gelar Pemain Terbaik Dunia FIFA, Semenjak bergabung dengan Barcelona pada tahun 2004. Ia juga merupakan pemain dengan jumlah gol tertinggi dalam sejarah Liga Spanyol dan salah satu top scorer sepanjang masa di Liga Champions UEFA.'

In [ ]:
conv_memory.run("dari bidang IT yang terpopuler tersebut, yang paling banyak di butuhkan di Indonesia bidang apa?")

' Di Indonesia, bidang Teknologi Informasi paling banyak dibutuhkan adalah pengembangan software, pemrograman, keamanan jaringan, teknologi cloud, dan analisis data. Bidang ini juga mencakup arsitektur dan desain informasi, analisis bisnis, pengembangan web, pengenalan citra digital, dan banyak lagi.'

In [ ]:
conv_memory.run("berapa pendapatan pemain bola tersebut?")

' Saya tidak tahu berapa pendapatan pemain bola. Namun, menurut laporan terbaru, gaji rata-rata seorang pemain profesional di liga bola sepak profesional di Indonesia rata-rata sekitar Rp 30 juta per musim.'

Terlihat bahwasanya percakapan tentang pemain bola tersebut tidak lagi diketahui lagi oleh LLM. hal ini dikarenakan telah dibatasi untuk 1 percakapan saja ingatanynya.

In [4]:
(prompt := "data")

'data'

In [2]:
prompt

'data'